In [11]:
import pandas as pd
import os
import openpyxl
import numpy as np
import re
from datetime import datetime
import requests

In [426]:
class HelperFunctions():
    #input: a list of quarters such as "Q2 2022"
    #iterates through a finds a date/year in the file
    def getYears(self,quarters): 
        return [int(re.findall("\d{4}",quarter)[0]) for quarter in quarters]
    
    def getUniqueYears(self,quarters):
        return set(self.getYears(quarters))
    
    #input: compiledStatement
    #returns all quarters but removes first col of label
    def getQuarters(self, compiledStatement): 
        return list(compiledStatement.columns[1:])
    
    def getPossibleFilingsFromYears(self, years): 
        return [q + " " + str(year) for year in years for q in ["Q1","Q2","Q3","Q4","Year Ended"]]
    
    #input: Excel dictionary with sheet name and the sheet dataframe
    #finds all possible filing names post fromDate and filters through dictionary keys to find corresponding filling
    def getExcelSheetsPostDate(self,Excel):  
        years = range(self.fromDate,self.endDate+1)
        names = self.getPossibleFilingsFromYears(years)
        #names = ["Year Ended "+str(self.fromDate-1)]+names
        files = [filing for name in names for filing in Excel.keys() if name in filing] 
        Excel = {file:Excel[file] for file in files}
        return Excel
    
    #input: compiledStatement
    #find all years in statement and get all possible filings in order then only keep the ones in actually in the statement
    def reorderQuarters(self,compiledStatement): 
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        cols = self.getPossibleFilingsFromYears(years)
        cols = [col for col in cols if col in quarters]
        compiledStatement = compiledStatement[["label"] + cols]
        return compiledStatement
    
    
    def cleanup_label(self, label):
        new_label = label.lower()
        new_label = re.sub("\([\w\W]+?\)","",new_label) #replace parenthesis wtih nothing
        new_label = re.sub(" \$|\s*\d+\,*", "", new_label) #replace money amounts in label with nothing
        new_label = new_label.replace("gain","loss").replace("decrease","increase").replace("  "," ").replace("—"," - ").replace("/","").strip()
        return new_label
    
    
    #input: Excel dictionary with sheet name and the sheet dataframe
    #go through each possible sheet and merge into a compiledStatement
    #to do this I use an iterative approach where I start from a completely empty dataframe and then go through each row of compiledStatement and sheet based on index. 
    #if the label match, i add data from both. if the labels don't match then I will by default add the previously compiledStatement and continue moving
    def addData(self, Excel):
        compiledStatement = pd.DataFrame(columns=["label"])
        excel = list(Excel.items())
        excel.reverse()
        for quarter,sheet in excel:
            quarter =  re.sub("\s\([\w\W]+?\)","",quarter) #remove the parenthesis ie. (2021) from the quarter
            sheet.columns = ["label",quarter]
            
            sheet = sheet.dropna(subset="label", axis=0, how="any")
            sheet = sheet[(~sheet[quarter].astype("string").str.contains("[a-zA-Z]",regex=True))|sheet[quarter].isnull()] #remove rows where the numerical data is a string ie. "Hotel"
            sheet.label = sheet.label.apply(self.cleanup_label) #normalize the label
            
            sheeti = statementi = 0
            newStatement = pd.DataFrame()
            
            while sheeti < len(sheet) and statementi < len(compiledStatement):
            #merge the current compiledStatement and new sheet into one statement 
            #go through each row of the two statements via index; stop when reach bottom of one
            
                sheet_label = sheet.iloc[sheeti].label
                statement_label = compiledStatement.iloc[statementi].label
                newRowStatement = compiledStatement.iloc[statementi:statementi+1].reset_index(drop=True) #grab the row and drop the index so that they merge
                newRowSheet = sheet.iloc[sheeti:sheeti+1].reset_index(drop=True)

                #check if the labels are the same 
                if statement_label == sheet_label:
                    #if labels are the same then create a new row by concating the rows
                    newRowSheet = newRowSheet.drop(columns="label")
                    newRow = pd.concat([newRowStatement,newRowSheet],axis=1)
                    newStatement = pd.concat([newStatement, newRow], ignore_index=True)
                    
                    #move on to the next row
                    sheeti+=1
                    statementi+=1
                    
                else:
                    maxi = min(sheeti+5,len(sheet))
                    
                    #next, before I append the final statement label, I want to check if the statement label corresponds to the next label for the sheet
                    #if it corresponds, then I will move the sheet up instead
                    if statement_label in list(sheet.iloc[sheeti:maxi].label): 
                        newRowSheet["value"] = sheeti #since a new label is being added, then add a value to the row to keep track of its original index
                        newStatement = pd.concat([newStatement,newRowSheet]) #concat the sheet row to the new statement
                        sheeti += 1
                        continue
                    
                    #if the two labels don't match, default is to add the row of the final statment
                    newStatement = pd.concat([newStatement,newRowStatement])
                    statementi += 1
        
            #if either sheet hasn't finished, then add the end of the statement to the new one
            sheet["value"] = sheet.index
            newStatement = pd.concat([newStatement,sheet.iloc[sheeti:len(sheet)]])
            newStatement = pd.concat([newStatement,compiledStatement.iloc[statementi:len(compiledStatement)]])

            compiledStatement = newStatement
        
        compiledStatement = compiledStatement.sort_values(by="value").drop(columns="value") #value is record of the order of the label in the original sheet
        compiledStatement = compiledStatement.reset_index(drop=True)
        return compiledStatement
    

In [427]:
class CompileIncomeStatement(HelperFunctions):
    #input: start date, end date, ticker
    #this class grabs all of the income statements from excel, filters them according to date, then it compiles each individual sheet into one statement
    def __init__(self,ticker,fromDate, endDate=2022):
        self.ticker = ticker
        self.fromDate = fromDate
        self.endDate = endDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.input = os.path.join(self.path, f"Income Statements All-{self.ticker}.xlsx")
        self.Excel = self.readExcel()
        self.Excel = self.getExcelSheetsPostDate(self.Excel)
        
        self.compiledStatement = self.addData(self.Excel)
        self.compiledStatement = self.performIncomeMath(self.compiledStatement)
        self.compiledStatement = self.reorderQuarters(self.compiledStatement)
    
    def readExcel(self):
        Excel = pd.read_excel(self.input,sheet_name=None)
        return Excel
    
    #input: compiledStatement
    #perform math to find q4 of the income statement which is the total year end minus q3
    def performIncomeMath(self,compiledStatement): 
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        
        for year in years:
            year = str(year)
            if "Year Ended "+year in quarters and "Q3 "+year in quarters: 
                compiledStatement["Q4 "+year] = compiledStatement["Year Ended "+year] - compiledStatement["Q3 "+year]
                compiledStatement.loc[compiledStatement.label.str.startswith("weighted-average"),"Q4 "+year] = compiledStatement.loc[compiledStatement.label.str.startswith("weighted-average"),"Year Ended "+year]
        return compiledStatement

            

In [428]:
class Income:
    #input ticker, start date
    #creates one massive compiled statement with all filings from the start date
    #then creates a bunch of income statements compiled by year where filings are "more" normalized placed side by side that can be manually concated
    def __init__(self,ticker,fromDate):
        self.ticker = ticker
        self.fromDate = fromDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.output = os.path.join(self.path, f"Compiled Income Statement-{self.ticker}.xlsx")
        
        self.createSeparateStatements()
        self.createStatement()
        
        self.writeExcel()
        
    #input: start date
    #iterate through each possible year and create compiled statement for that fiscal year
    #i found that the statements tend to be more normalized within one year so this strategy can prevent out of control long compiled statements
    def createSeparateStatements(self): 
        self.separateCompiledStatement = pd.DataFrame()
        for year in range(self.fromDate,2023):
            statement = CompileIncomeStatement(self.ticker, year,year)
            self.separateCompiledStatement = pd.concat([self.separateCompiledStatement,statement.compiledStatement],axis=1)
    
    def createStatement(self):
        self.compiledStatement = CompileIncomeStatement(self.ticker, self.fromDate).compiledStatement
        
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.output)
        
        self.compiledStatement.to_excel(Excel, index=False, sheet_name="Compiled Statement")
        self.separateCompiledStatement.to_excel(Excel, index=False, sheet_name="Separately Compiled Statement")
        
        Excel.save()
         
    

In [429]:
income = Income("TSLA",2017)
income.compiledStatement



,label,Year Ended 2017,Q1 2018,Q2 2018,Q3 2018,Q4 2018,Year Ended 2018,Q1 2019,Q2 2019,Q3 2019,...,Q3 2020,Q4 2020,Year Ended 2020,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Year Ended 2021,Q1 2022,Q2 2022
0,revenues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,revenues,NaN,NaN,NaN,6824413.0,14636855.0,21461268.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,automotive sales,8534752.0,2561881.0,3117865.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,automotive leasing,1106548.0,173436.0,239816.0,220461.0,663000.0,883461.0,215120.0,208362.0,221.0,...,265.0,787.0,1052.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,total revenues,11758751.0,3408751.0,4002231.0,6824413.0,14636855.0,21461268.0,4541464.0,6349676.0,6303.0,...,8771.0,22765.0,31536.0,10389.0,11958.0,13757.0,40066.0,53823.0,18756.0,16934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,energy generation and storage segment [member],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103,cost of revenues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,cost of revenues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3906.0,NaN,NaN
105,total revenues,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,494.0,801.0,806.0,NaN,NaN,NaN,NaN


In [416]:
class CompileCashflowStatement(HelperFunctions):
    def __init__(self,ticker,fromDate,endDate=2022):
        self.ticker = ticker
        self.fromDate = fromDate
        self.endDate = endDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.input = os.path.join(self.path, f"Cashflow Statements All-{self.ticker}.xlsx")
        self.output = os.path.join(self.path, f"Compiled Cashflow Statement-{self.ticker}.xlsx")
        
        
        self.Excel = self.readExcel()
        self.Excel = self.getExcelSheetsPostDate(self.Excel)
        
        self.compiledStatement = self.addData(self.Excel)
        self.compiledStatement = self.performCashflowMath(self.compiledStatement)
        self.compiledStatement = self.reorderQuarters(self.compiledStatement)

        
    def readExcel(self):
        Excel = pd.read_excel(self.input,sheet_name=None)
        return Excel
    
    #input: compiledStatement
    #perform math to find q2,q3,q4 of the income statement because filings only contain data for past 9mo and past 6mo
    def performCashflowMath(self, compiledStatement):
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        
        #Q2 and Q3 are deceptively 6mo and 9mo 
        quarters = [quarter.replace("Q2","6mo").replace("Q3","9mo") for quarter in quarters]
        compiledStatement.columns = ["label"] + quarters
        
        #find labels that contain "cash at beginning of period" and "cash at end of period" 
        #the cash at beginning of period is not the different in cash between (for ex) 6mo and 9mo 
        begin_period = [label for label in compiledStatement.label if "beginning of period" in label and "cash" in label]
        end_period = [label for label in compiledStatement.label if "end of period" in label and "cash" in label]
        
        compiledStatement.index = compiledStatement.label

        for year in years:
            year = str(year)
            if "6mo "+year in quarters and "Q1 "+year in quarters: 
                # find Q2 
                compiledStatement["Q2 "+year] = compiledStatement["6mo "+year] - compiledStatement["Q1 "+year]
                # for ex. cash at beginning of period at Q2 is the same as cash at end of period for Q1
                compiledStatement.loc[begin_period,"Q2 "+year] = compiledStatement.loc[end_period,"Q1 "+year]
                compiledStatement.loc[end_period,"Q2 "+year] = compiledStatement.loc[end_period,"6mo "+year]

            if "9mo "+year in quarters and "6mo "+year in quarters: 
                #find Q3 
                compiledStatement["Q3 "+year] = compiledStatement["9mo "+year] - compiledStatement["6mo "+year]
                compiledStatement.loc[begin_period,"Q3 "+year] = compiledStatement.loc[end_period, "6mo "+year]
                compiledStatement.loc[end_period,"Q3 "+year] = compiledStatement.loc[end_period,"9mo "+year]

            if "Year Ended "+year in quarters and "9mo "+year in quarters: 
                #find Q3 
                compiledStatement["Q4 "+year] = compiledStatement["Year Ended "+year] - compiledStatement["9mo "+year]
                compiledStatement.loc[begin_period,"Q4 "+year] = compiledStatement["9mo "+year].loc[end_period]
                compiledStatement.loc[end_period,"Q4 "+year] = compiledStatement.loc[end_period,"Year Ended "+year]
        
        #remove 6mo and 9mo columns
        cols = [col for col in compiledStatement.columns if "9mo" not in col and "6mo" not in col]
        compiledStatement = compiledStatement[cols] 
        
        compiledStatement = compiledStatement.reset_index(drop=True)
        return compiledStatement
    
   
            

In [417]:
CompileCashflowStatement("GPS",2022).compiledStatement

,label,Q1 2022,Q2 2022,Q3 2022,Q4 2022,Year Ended 2022
0,cash flows from operating activities:,NaN,NaN,NaN,NaN,NaN
1,net income,166.0,258.0,-152.0,-16.0,256.0
2,adjustments to reconcile net income to net cas...,NaN,NaN,NaN,NaN,NaN
3,depreciation and amortization,120.0,124.0,128.0,132.0,504.0
4,share-based compensation,36.0,36.0,25.0,42.0,139.0
...,...,...,...,...,...,...
71,"cash, cash equivalents, and restricted cash en...",NaN,NaN,NaN,902.0,902.0
72,cash flow non cash investing disclosure [abstr...,NaN,NaN,NaN,NaN,NaN
73,purchases of property and equipment not yet pa...,NaN,NaN,NaN,NaN,124.0
74,cash paid for interest during the period,NaN,NaN,NaN,NaN,180.0


In [419]:
class Cashflow:
    def __init__(self,ticker,fromDate):
        self.ticker = ticker
        self.fromDate = fromDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.output = os.path.join(self.path, f"Compiled Cashflow Statement-{self.ticker}.xlsx")
        
        self.createSeparateStatements()
        self.createStatement()
        
        self.writeExcel()
        
    def createSeparateStatements(self):
        self.separateCompiledStatement = pd.DataFrame()
        for year in range(self.fromDate,2023):
            statement = CompileCashflowStatement(self.ticker, year,year)
            self.separateCompiledStatement = pd.concat([self.separateCompiledStatement,statement.compiledStatement],axis=1)
    
    def createStatement(self):
        self.compiledStatement = CompileCashflowStatement(self.ticker, self.fromDate).compiledStatement
        
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.output)
        self.compiledStatement.to_excel(Excel, index=False, sheet_name="Compiled Statement")
        self.separateCompiledStatement.to_excel(Excel, index=False, sheet_name="Separately Compiled Statement")
        
        Excel.save()
         
    

In [421]:
cashflow = Cashflow("MSFT",2019)
cashflow.compiledStatement



,label,Q1 2019,Q2 2019,Q3 2019,Q4 2019,Year Ended 2019,Q1 2020,Q2 2020,Q3 2020,Q4 2020,...,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Year Ended 2021,Q1 2022,Q2 2022,Q3 2022,Q4 2022,Year Ended 2022
0,operations,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
1,net income,8824.0,-404.0,389.0,30431.0,39240,10678.0,971.0,-897.0,33529.0,...,13893.0,1570.0,-6.0,45814.0,61271.0,20505.0,-1740.0,-2037.0,56010.0,72738
2,adjustments to reconcile net income to net cas...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
3,"depreciation, amortization, and other",2837.0,158.0,-69.0,8756.0,11682,2971.0,232.0,-85.0,9678.0,...,2645.0,116.0,175.0,8750.0,11686.0,3212.0,284.0,277.0,10687.0,14460
4,stock-based compensation expense,1107.0,76.0,-11.0,3480.0,4652,1262.0,78.0,-2.0,3951.0,...,1456.0,110.0,-41.0,4593.0,6118.0,1702.0,195.0,9.0,5596.0,7502
5,net recognized losss on investments and deriva...,-240.0,105.0,40.0,-697.0,-792,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-1249.0,-364.0,57.0,NaN,NaN,-409
6,net recognized losses on investments and deriv...,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,...,-128.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,deferred income taxes,-247.0,74.0,-147.0,-6143.0,-6463,-177.0,124.0,-153.0,217.0,...,-11.0,-6.0,-71.0,-62.0,-150.0,-5970.0,6153.0,-381.0,-5504.0,-5702
8,changes in operating assets and liabilities:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
9,accounts receivable,9194.0,-11590.0,2856.0,-3272.0,-2812,10090.0,-14293.0,5094.0,-3468.0,...,8843.0,-12851.0,4298.0,-6771.0,-6481.0,10486.0,-16029.0,6400.0,-7691.0,-6834


In [392]:
class CompileBalanceStatement(HelperFunctions):
    def __init__(self,ticker,fromDate,endDate=2022):
        self.ticker = ticker
        self.fromDate = fromDate
        self.endDate = endDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.input = os.path.join(self.path, f"Balance Statements All-{self.ticker}.xlsx")
        self.output = os.path.join(self.path, f"Compiled Balance Statement-{self.ticker}.xlsx")
        
        
        self.Excel = self.readExcel()
        self.Excel = self.getExcelSheetsPostDate(self.Excel)
        
        self.compiledStatement = self.addData(self.Excel)
        self.compiledStatement = self.performBalanceMath(self.compiledStatement)
        self.compiledStatement = self.reorderQuarters(self.compiledStatement)

        
    def readExcel(self):
        Excel = pd.read_excel(self.input,sheet_name=None)
        return Excel
    
    
    def performBalanceMath(self, compiledStatement):
        quarters = self.getQuarters(compiledStatement)
        years = self.getUniqueYears(quarters)
        
        for year in years:
            year = str(year)
            if "Year Ended "+year in quarters and "Q3 "+year in quarters: 
                compiledStatement["Q4 "+year] = compiledStatement["Year Ended "+year]
        return compiledStatement
    
   
            

In [393]:
class Balance:
    def __init__(self,ticker,fromDate):
        self.ticker = ticker
        self.fromDate = fromDate
        
        self.path = os.path.join("input",f"Financial Statement {self.ticker}")
        self.output = os.path.join(self.path, f"Compiled Balance Statement-{self.ticker}.xlsx")
        
        self.createSeparateStatements()
        self.createStatement()
        
        self.writeExcel()
        
    def createSeparateStatements(self):
        self.separateCompiledStatement = pd.DataFrame()
        for year in range(self.fromDate,2023):
            statement = CompileBalanceStatement(self.ticker, year,year)
            self.separateCompiledStatement = pd.concat([self.separateCompiledStatement,statement.compiledStatement],axis=1)
    
    def createStatement(self):
        self.compiledStatement = CompileBalanceStatement(self.ticker, self.fromDate).compiledStatement
        
    def writeExcel(self):
        Excel = pd.ExcelWriter(self.output)
        self.compiledStatement.to_excel(Excel, index=False, sheet_name="Compiled Statement")
        self.separateCompiledStatement.to_excel(Excel, index=False, sheet_name="Separately Compiled Statement")
        
        Excel.save()
         
    

In [394]:
balance = Balance("MSFT",2019)
balance.compiledStatement



,label,Q1 2019,Q2 2019,Q3 2019,Q4 2019,Year Ended 2019,Q1 2020,Q2 2020,Q3 2020,Q4 2020,...,Q1 2021,Q2 2021,Q3 2021,Q4 2021,Year Ended 2021,Q1 2022,Q2 2022,Q3 2022,Q4 2022,Year Ended 2022
0,current assets:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
1,cash and cash equivalents,15137,6638,11212,11356,11356,13117,8864,11710,13576,...,17205,14432,13702,14224,14224,19165,20604,12498,13931,13931
2,short-term investments,120743,121024,120406,122463,122463,123519,125389,125916,122951,...,120772,117536,111705,116110,116110,111450,104765,92195,90826,90826
3,"total cash, cash equivalents, and short-term i...",135880,127662,131618,133819,133819,136636,134253,137626,136527,...,137977,131968,125407,130334,130334,130615,125369,104693,104757,104757
4,"accounts receivable, net of allowance for doub...",17390,19680,19269,29524,29524,19087,23525,22699,32011,...,22851,27312,26322,38043,38043,27349,33520,32613,44261,44261
5,inventories,3614,1961,1951,2063,2063,2622,1823,1644,1895,...,2705,1924,2245,2636,2636,3411,3019,3296,3742,3742
6,other current assets,NaN,NaN,NaN,NaN,NaN,7551,7473,8536,11482,...,13544,12769,11640,13393,13393,12951,12280,13320,16924,16924
7,other,7311,7571,7049,10146,10146,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,total current assets,164195,156874,159887,175552,175552,165896,167074,170505,181915,...,177077,173973,165614,184406,184406,174326,174188,153922,169684,169684
9,"property and equipment, net of accumulated dep...",31430,32717,33648,36477,36477,38409,40522,41221,44151,...,47927,51737,54945,59715,59715,63772,67214,70298,74398,74398
